# 3.3 Transforming IFC to LBD graphs

Of course, after the earlier exercises, where we loaded CSV data and create RDF graphs, and where we create RDF graphs from scratch, including also ontologies and querying and inspecting them.... we also want to Extract - Transform - Load IFC files and turn them into LBD graphs. While there are multiple converters available, in the below scripts, you will find out how you can do this yourself and edit it such that this works for your case and sake.

## 1. IfcOpenShell
For this exercise, we will rely on `ifcopenshell` as the dedicated Python library to load and read IFC files. If you want to know more about `ifcopenshell`, use the documentation that you can find here: https://docs.ifcopenshell.org/ifcopenshell-python.html.


In [1]:
import ifcopenshell as ios

No stream support: No module named 'lark'


For this exercise, we will use the TUe Atlas BIM model, and try to transform it into an RDF LBD graph that can be used. This will lead to a bunch of Python code, that is incomplete of course, and can be completed and customized as preferred.

- [Atlas_8_floor.rvt](data/Atlas_8_floor.rvt)
- [Atlas_8_floor.ifc](data/Atlas_8_floor.ifc)
- [Atlas_8_floor.ttl](data/Atlas_8_floor.ttl)

Let's first load an IFC model in memory, using `ifcopenshell`, which is here abbreviated as `ios`.

In [4]:
model = ios.open("data/Atlas_8_floor.ifc")

Second, we prepare an `outputFile` where we can store the output. Below, this file is opened for writing. Don't forget to close the file when you are done, using `file.close()`.

In [11]:
outputfile = open("output/outputFile.ttl", "w")

Note that in the below code, we will write content directly to a file, without using any library other than `ifcopenshell` for reading the IFC content. Of course, it is also possible to rely on `rdflib` for writing the output. Also, one can directly output into a database, instead of passing by a file, using the methods shown earlier.

## 2. Writing the header with namespace information
First content to be created in the `TTL` output is the header with all namespaces information. For that purpose, let's first some namespace variables that contain namespace strings that we will use frequently. 

In [8]:
RDF = "https://www.w3.org/1999/02/22-rdf-syntax-ns#"
RDFS = "https://www.w3.org/2000/01/rdf-schema#"
OWL = "https://www.w3.org/2002/07/owl#"
XSD = "https://www.w3.org/2001/XMLSchema#"
DCE = "https://purl.org/dc/elements/1.1/"
VANN = "https://purl.org/vocab/vann/"
CC = "https://creativecommons.org/ns#"
BOT = "https://w3id.org/bot#"
BEO = "https://pi.pauwel.be/voc/buildingelement#"
MEP = "https://pi.pauwel.be/voc/distributionelement#"
GEOM = "https://w3id.org/geom#"
PROPS = "https://w3id.org/props#"

from datetime import datetime
now = datetime.now()
current_time = now.strftime('%Y%m%d_%H%M%S')
baseURI = "https://linkedbuildingdata.net/ifc/resources" + current_time + "/"
print("baseURI : " + baseURI)

baseURI : http://linkedbuildingdata.net/ifc/resources20240610_070350/


We then create a string that contains all the header information that we need. This string is then written to a file. Thus, we open a file in `write` mode (`w`), we then write to it, and we close it again. We can also open a file in `append` mode (`a`). Be careful what you use when. 

In [16]:
# creating a string to write
s = "# baseURI: " + baseURI + "\n"
s+= "@prefix inst: <" + baseURI + "> .\n"
s+= "@prefix rdf:  <" + RDF + "> .\n"
s+= "@prefix rdfs:  <" + RDFS + "> .\n"
s+= "@prefix xsd:  <" + XSD + "> .\n"
s+= "@prefix bot:  <" + BOT + "> .\n"
s+= "@prefix beo:  <" + BEO + "> .\n"
s+= "@prefix mep:  <" + MEP + "> .\n"
s+= "@prefix geom:  <" + GEOM + "> .\n"
s+= "@prefix props:  <" + PROPS + "> .\n\n"

s+= "inst: rdf:type <https://www.w3.org/2002/07/owl#Ontology> .\n\n"

outputfile = open("output/outputFile.ttl", "w")
outputfile.write(s)
outputfile.close()

You can always restart file writing by emptying the file, as follows:

In [14]:
open('output/outputfile.ttl', 'w').close()

## 3. Writing out the sites in the IFC file.
We will start with something straightforward, namely the `IfcSite` elements in the model. These have to be parsed, and then written to the `TTL` file as `bot:Site` instances.

In [18]:
output = ""
for s in model.by_type("IfcSite"):                
    output += "inst:site_"+str(s.id()) + "\n"
    output += "\ta bot:Site ;" + "\n"
    if(s.Name):
        output += "\trdfs:label \""+s.Name+"\"^^xsd:string ;" + "\n"
    if(s.Description):
        output += "\trdfs:comment \""+s.Description+"\"^^xsd:string ;" + "\n"        
    output += "\tbot:hasGuid \""+ ios.guid.expand(s.GlobalId) +"\"^^xsd:string ;" + "\n"
    output += "\tprops:hasCompressedGuid \""+ s.GlobalId +"\"^^xsd:string "
    for reldec in s.IsDecomposedBy:
        if reldec is not None:
            for b in reldec.RelatedObjects:
                output += ";\n"
                output += "\tbot:hasBuilding inst:building_"+ str(b.id()) + " "
    output += ". \n\n"

#return output
outputfile = open("output/outputFile.ttl", "a")
outputfile.write(output)
outputfile.close()

There we go, we have ourselves a site in the RDF graph! Let's add some more elements!

## 4. Adding some properties
While the site looks nice in RDF, we are missing the properties that it has. We will need those properties, also for other elements in the graph. So let's try to add those first. Since we will do this a couple of times, let's create appropriate functions to do this. These should clearly be tested and expanded for proper use in a larger setting.

In the below function, we assume that this is executed as part of a method that writes out an element in RDF already. The content is therefore appended to an existing `subject` node in RDF, and it lacks this first `subject` element as well as the closing dot (`.`). Try it to understand it.

In [19]:
def cleanString(name):
    name = ''.join(x for x in name.title() if not x.isspace())
    name = name.replace('\\', '')
    name = name.replace('/', '')
    return name

def print_properties(properties, output):    
    for name, value in properties.items():   
        if name == "id":
            continue     
        name = cleanString(name)
        output += ";\n"
        if isinstance(value, int):          
            output += "\tprops:"+name+" \""+ str(value) +"\"^^xsd:int "
        elif isinstance(value, float):    
            output += "\tprops:"+name+" \""+ str(value) +"\"^^xsd:double "
        else:           
            output += "\tprops:"+name+" \""+ str(value) +"\"^^xsd:string "
    return output

Let's try our function and write some site properties.

In [20]:
output = ""
for s in model.by_type("IfcSite"): 
    site_psets = ios.util.element.get_psets(s)
    for name, properties in site_psets.items():
        output = print_properties(properties, output)

outputfile = open("output/outputFile.ttl", "a")
outputfile.write(output)
outputfile.close()

When inspecting the output file, we clearly see incorrect output. This TTL is corrupt and not usable as such. This is what we anticipated. You can see in the below printout what went wrong. The content needs to be *inside* the block that describes the site, not outside this block.

<img src="figures/wrongsiteTTL.png" width="600" />

So we will clear our file content, write the header again, and rewrite the correct file content: site with properties.

In [22]:
# creating a string to write
s = "# baseURI: " + baseURI + "\n"
s+= "@prefix inst: <" + baseURI + "> .\n"
s+= "@prefix rdf:  <" + RDF + "> .\n"
s+= "@prefix rdfs:  <" + RDFS + "> .\n"
s+= "@prefix xsd:  <" + XSD + "> .\n"
s+= "@prefix bot:  <" + BOT + "> .\n"
s+= "@prefix beo:  <" + BEO + "> .\n"
s+= "@prefix mep:  <" + MEP + "> .\n"
s+= "@prefix geom:  <" + GEOM + "> .\n"
s+= "@prefix props:  <" + PROPS + "> .\n\n"

s+= "inst: rdf:type <http://www.w3.org/2002/07/owl#Ontology> .\n\n"

outputfile = open("output/outputFile.ttl", "w")
outputfile.write(s)
outputfile.close()

In [23]:
output = ""
for s in model.by_type("IfcSite"):                
    output += "inst:site_"+str(s.id()) + "\n"
    output += "\ta bot:Site ;" + "\n"
    if(s.Name):
        output += "\trdfs:label \""+s.Name+"\"^^xsd:string ;" + "\n"
    if(s.Description):
        output += "\trdfs:comment \""+s.Description+"\"^^xsd:string ;" + "\n"        
    output += "\tbot:hasGuid \""+ ios.guid.expand(s.GlobalId) +"\"^^xsd:string ;" + "\n"
    output += "\tprops:hasCompressedGuid \""+ s.GlobalId +"\"^^xsd:string "
    for reldec in s.IsDecomposedBy:
        if reldec is not None:
            for b in reldec.RelatedObjects:
                output += ";\n"
                output += "\tbot:hasBuilding inst:building_"+ str(b.id()) + " "    
    site_psets = ios.util.element.get_psets(s)
    for name, properties in site_psets.items():
        output = print_properties(properties, output)
    output += ". \n\n"

#return output
outputfile = open("output/outputFile.ttl", "a")
outputfile.write(output)
outputfile.close()

Let's check our output again. This looks a lot better!

<img src="figures/correctsiteTTL.png" width="600" />

## 5. Writing buildings, storeys, spaces, and a range of elements
We got the basics; the rest is just 'more of the same', which does not mean that it is easy, as all information mainly needs to be parsed correctly from the IFC file and transformed appropriately into an RDF graph that follows a meaningful set of ontologies. This is nevertheless not that different from any other Extract-Transform-Load (ETL) procedure. 

In the below snippets, we will add more elements, using the same methods that we used before. Actually, let's try to write some methods that we can re-use more easily.

In [24]:
def writeBuildings(model):
    output = ""
    for b in model.by_type("IfcBuilding"):                
        output += "inst:building_"+str(b.id()) + "\n"
        output += "\ta bot:Building ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"        
        output += "\tbot:hasGuid \""+ ios.guid.expand(b.GlobalId) +"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "
        for reldec in b.IsDecomposedBy:
            if reldec is not None:
                for st in reldec.RelatedObjects:
                    output += ";\n"
                    output += "\tbot:hasStorey inst:storey_"+ str(st.id()) + " "
        psets = ios.util.element.get_psets(b)
        for name, properties in psets.items():
            output = print_properties(properties, output)                             
                
        output += ". \n\n"
    return output

In [25]:
def writeStoreys(model):
    output = ""
    for b in model.by_type("IfcBuildingStorey"):                
        output += "inst:storey_"+str(b.id()) + "\n"
        output += "\ta bot:Storey ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"        
        output += "\tbot:hasGuid \""+ ios.guid.expand(b.GlobalId) +"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "
        for reldec in b.IsDecomposedBy:
            if reldec is not None:
                for st in reldec.RelatedObjects:
                    output += ";\n"
                    output += "\tbot:hasSpace inst:space_"+ str(st.id()) + " "
        for relcontains in b.ContainsElements:
            if relcontains is not None:
                for st in relcontains.RelatedElements:
                    output += ";\n"
                    output += "\tbot:containsElement inst:element_"+ str(st.id()) + " "

        psets = ios.util.element.get_psets(b)
        for name, properties in psets.items():
            output = print_properties(properties, output)                             
                
        output += ". \n\n"
    return output

In [32]:
def writeSpaces(model):
    output = ""
    for b in model.by_type("IfcSpace"):                
        output += "inst:space_"+str(b.id()) + "\n"
        output += "\ta bot:Space ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"        
        output += "\tbot:hasGuid \""+ ios.guid.expand(b.GlobalId) +"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "
        for relbounded in b.BoundedBy:
            if relbounded is not None:
                st = relbounded.RelatedBuildingElement
                if st is not None:
                    output += ";\n"
                    output += "\tbot:adjacentElement inst:element_"+ str(st.id()) + " "
        for relcontains in b.ContainsElements:
            if relcontains is not None:
                counter = 0
                for st in relcontains.RelatedElements:
                    if counter == 0:
                        output += ";\n"
                        output += "\tbot:containsElement inst:element_"+ str(st.id()) + " "
                        counter+=1
                    else:
                        output += ", inst:element_"+ str(st.id()) + " "
                        counter+=1

        psets = ios.util.element.get_psets(b)
        for name, properties in psets.items():
            output = print_properties(properties, output)                    
                
        output += ". \n\n"
    return output

In [27]:
def writeElements(model):
    output = ""
    for b in model.by_type("IfcElement"):                
        output += "inst:element_"+str(b.id()) + "\n"
        output += "\ta bot:Element ;" + "\n"
        if(b.Name):
            output += "\trdfs:label \""+b.Name+"\"^^xsd:string ;" + "\n"
        if(b.Description):
            output += "\trdfs:comment \""+b.Description+"\"^^xsd:string ;" + "\n"        
        output += "\tbot:hasGuid \""+ ios.guid.expand(b.GlobalId) +"\"^^xsd:string ;" + "\n"
        output += "\tprops:hasCompressedGuid \""+ b.GlobalId +"\"^^xsd:string "

        for relvoids in b.HasOpenings:
            if relvoids is not None:
                st = relvoids.RelatedOpeningElement
                for relfills in st.HasFillings:
                    if relfills is not None:
                        filler = relfills.RelatedBuildingElement
                        output += ";\n"
                        output += "\tbot:hostsElement inst:element_"+ str(filler.id()) + " "

        for relvoids in b.HasOpenings:
            if relvoids is not None:
                st = relvoids.RelatedOpeningElement
                for relfills in st.HasFillings:
                    if relfills is not None:
                        filler = relfills.RelatedBuildingElement
                        output += ";\n"
                        output += "\tbot:hostsElement inst:element_"+ str(filler.id()) + " "

        psets = ios.util.element.get_psets(b)
        for name, properties in psets.items():
            output = print_properties(properties, output)                             
                
        output += ". \n\n"
    return output

Let's see where these functions will bring us. In principle, they can be used for printing out all buildings, storeys, spaces, and elements in reasonable detail. Let's execute them one by one to see what is added to the output file. 

In [28]:
# be careful, this filewriter is set to append mode. Running this again and again will add the same information over and over again.
outputfile = open("output/outputFile.ttl", "a")
outputString = writeBuildings(model)
outputfile.write(outputString)
outputfile.close()

In [29]:
# storeys
outputfile = open("output/outputFile.ttl", "a")
outputString = writeStoreys(model)
outputfile.write(outputString)
outputfile.close()

In [33]:
# spaces
outputfile = open("output/outputFile.ttl", "a")
outputString = writeSpaces(model)
outputfile.write(outputString)
outputfile.close()

**Note: executing this function can take a long time** when doing this with a model that has plenty of elements, and even more properties. There are a number of ways in which you can improve this.

In [34]:
# elements
outputfile = open("output/outputFile.ttl", "a")
outputString = writeElements(model)
outputfile.write(outputString)
outputfile.close()

## 6. Future work 
We now have the basic structure of an IFC file transformed into an RDF file. 

Further additions that can be made, are:
- adding zoning information
- improving the way in which properties are stored
- adding geometry representations
- adding materials and quantities
